In [1]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import seaborn as sns
import torch
import torch.nn as nn
import torchvision
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

device = "cuda" if torch.cuda.is_available() else "cpu"
device


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'cpu'

In [2]:
df = pd.read_csv("./dataDinamico10ClassHD.csv", delimiter=',')   # Leemos el dataframe origianal con puntos en fila por frame
# df = pd.read_csv("./dataDinamico.csv", delimiter=',')   # Leemos el dataframe origianal con puntos en fila por frame
df

,palabra,IDv,Derecha_0_x,Derecha_0_y,Derecha_0_z,Derecha_1_x,Derecha_1_y,Derecha_1_z,Derecha_2_x,Derecha_2_y,...,Izquierda_17_z,Izquierda_18_x,Izquierda_18_y,Izquierda_18_z,Izquierda_19_x,Izquierda_19_y,Izquierda_19_z,Izquierda_20_x,Izquierda_20_y,Izquierda_20_z
0,Hola,0,0.605908,0.368811,1.405228e-07,0.582437,0.367107,-0.002988,0.569147,0.343845,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Hola,0,0.607985,0.381254,2.055655e-07,0.582216,0.374213,-0.000726,0.566853,0.347438,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Hola,0,0.612550,0.388733,1.701401e-07,0.586914,0.380896,-0.001106,0.569448,0.353957,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Hola,0,0.616269,0.389765,1.606986e-07,0.589837,0.382550,-0.000686,0.571279,0.354775,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Hola,0,0.623996,0.386217,1.261458e-07,0.596504,0.379922,-0.001194,0.577093,0.350606,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4563,PorFavor,302,0.419028,0.684288,1.446481e-07,0.436740,0.640336,-0.004582,0.459739,0.618643,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4564,PorFavor,302,0.407910,0.658527,1.575484e-07,0.425568,0.618243,-0.004666,0.447479,0.593829,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4565,PorFavor,302,0.403243,0.636015,1.421940e-07,0.419755,0.594429,-0.004092,0.441105,0.567604,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4566,PorFavor,302,0.405637,0.622480,1.321479e-07,0.419594,0.578779,-0.002542,0.441157,0.552077,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
# Extraemos las clases a clasificar
classes = df['palabra'].unique()   # Total clases a clasificar
classes

array(['Hola', 'BuenasTardes', 'BuenosDias', 'Gracias', 'Chau', 'Puedo',
       'BuenasNoches', 'Permiso', 'Saludos', 'PorFavor'], dtype=object)

In [4]:
# Contamos el numero de frames por gesto y verificamos el gesto con mayor longitud de frames
cantidaFrames = pd.DataFrame()
cantidaFrames = df['IDv'].value_counts() # frames por cada gesto o video de gesto
cantidaMaximaFrames = cantidaFrames.max()
cantidadVideos = len(cantidaFrames)
cantidaFrames,cantidaMaximaFrames, cantidadVideos 
# el video o gesto con frames mas larga

(283    26
 282    25
 285    25
 143    24
 30     24
        ..
 3       6
 23      6
 7       5
 2       5
 9       4
 Name: IDv, Length: 303, dtype: int64,
 26,
 303)

In [5]:
def flattenOneOrAllSign(numClasses, dataFrame):
    contadorClase = -1  # El contador inicia en cero del indice del dataframe 
    matriz_X_data = []  # Declaramos matriz vacia para cargar los gestos aplanados  -> vector
    gestoLargo = cantidaMaximaFrames * (dataFrame.shape[1]-2)     # frames del gesto mas largo
    print(gestoLargo)
    for i,j in enumerate(range(numClasses)):
        df_mask = dataFrame['IDv'].values == i # condicion para filtrar de una columna
        dataFrameAux = dataFrame[df_mask]      # pasar al dataframe la condicion que filtra todos los frames de un gesto
        positions = np.flatnonzero(df_mask)     
        dataFrameAux = dataFrame.iloc[positions]
        dataFrameAux = dataFrameAux.iloc[:, 2:]           # Quitamos la clse y el ID del del gesto del dataset
        gestoAplanado = dataFrameAux.to_numpy().flatten() # convertir a un np.array y aplanar todos los frames de un gesto
        contadorClase += len(dataFrameAux)                # Sumamos en indice de la clase para agregar una vez al gesto aplanado
        cerosFaltante = gestoAplanado.shape[0]            # capturamos las dimensiones del gesto para completar al gesto mas largo
        rowZeros = np.zeros(gestoLargo-cerosFaltante)     # generamos vector con ceros para aumentar
        rowZeros = np.append(rowZeros,dataFrame['palabra'][contadorClase])
        gestoAplanado = np.append(gestoAplanado, rowZeros, axis=0)  # agregar ceros en fila si el gesto de es menos frames axis = 0
        matriz_X_data.append(gestoAplanado)               # Agregamos a la matriz el gesto aplanado
    return pd.DataFrame(matriz_X_data)

dataFrameX = flattenOneOrAllSign(cantidadVideos,df)
dataFrameX

3276


,0,1,2,3,4,5,6,7,8,9,...,3267,3268,3269,3270,3271,3272,3273,3274,3275,3276
0,0.6059078574180603,0.3688112199306488,1.405227862960601e-07,0.5824374556541443,0.3671073615550995,-0.0029877058696001,0.5691474676132202,0.3438445031642914,-0.0054817125201225,0.5606343746185303,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Hola
1,0.3882864117622375,0.4419835209846496,1.5400065223047932e-08,0.4055489301681518,0.443995326757431,0.0089096324518322,0.419310986995697,0.4220169186592102,0.017609853297472,0.427785187959671,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Hola
2,0.3508655726909637,0.3901086747646332,7.831241077838058e-08,0.3778797090053558,0.3930847942829132,0.0028700646944344,0.4004311859607696,0.3592833280563354,0.0096158869564533,0.4140465855598449,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Hola
3,0.3614401221275329,0.3442257046699524,9.052937599562939e-08,0.3867232799530029,0.3532943725585937,0.0025240432005375,0.4058367908000946,0.3354128897190094,0.0085563017055392,0.417798638343811,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Hola
4,0.3969780206680298,0.3959761559963226,6.524541618091463e-08,0.4153926968574524,0.4007662534713745,0.0052622221410274,0.4299236834049225,0.383524477481842,0.0113708144053816,0.4397577941417694,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Hola
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,0.3832443952560425,0.5512071251869202,1.3878644722353786e-07,0.3918788433074951,0.5102128982543945,-0.0007970137521624,0.4095282256603241,0.48842453956604,-0.0024673985317349,0.4244277775287628,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,PorFavor
299,0.4293983578681946,0.5368232727050781,1.4058304032005253e-07,0.4347773790359497,0.4976741671562195,-0.0034709984902292,0.4472302794456482,0.477679580450058,-0.0067139188759028,0.4578877687454223,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,PorFavor
300,0.4351881444454193,0.5330913662910461,1.6524235491033326e-07,0.4379990398883819,0.4931097328662872,-0.00473841605708,0.451113224029541,0.4668866991996765,-0.0083860680460929,0.4636361598968506,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,PorFavor
301,0.4291908740997314,0.5514005422592163,1.416685648791827e-07,0.433257520198822,0.5109119415283203,-0.0020788400433957,0.4486841261386871,0.48494952917099,-0.0037177507765591,0.4609803557395935,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,PorFavor


In [7]:
# GUARDAR EL DATAFRAME EN UN ARCHIVO CSV
dataFrameX.to_csv("./DatasetDinamicoAplanado10ClassHD.csv", index= False)